# Sammeln der Expose-Links

In [18]:
import re
import time
from itertools import chain
from collections import OrderedDict
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
url = 'https://www.immowelt.de/liste/hamburg/haeuser/kaufen?sort=relevanz'
driver = webdriver.Firefox()
def soup_get(url):
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    html = driver.execute_script('return document.documentElement.outerHTML')
    
    return BeautifulSoup(html, 'html.parser')

sel_soup = soup_get(url)
#print(len(sel_soup.findAll("img")))

In [3]:
def href_finder(soup_ele):
    return [a['href'] for a in soup_ele.findAll('a',href=True)]
    
hrefs = href_finder(sel_soup)

In [4]:
# get number of pages
cps = [re.findall('cp=(\w+)',a) for a in hrefs]
cps = list(filter(None,cps))
num_pages = max([int(b[0]) for b in cps])

In [5]:
def href_extr(hrefs):
    """
    Takes soup element of one search page of immowelt and returns
    all the hrefs as list of texts
    """
    exposes = [re.findall('\/expose\/(\w+)',a) for a in hrefs]
    exposes = [a[0] for a in exposes if len(a)!=0]
    exposes = list(OrderedDict.fromkeys(exposes))
    return exposes

In [6]:
def projekt_finder(hrefs):
    projekts = [re.findall('\/projekte\/expose\/(\w+)',a) for a in hrefs]
    #
    projekts = list(filter(None,projekts))
    projekts = [*projekts]
    #projekts = list(OrderedDict.fromkeys(projekts))
    return list(set(chain.from_iterable(projekts)))

In [7]:
Exposes = list()
Projekte = list()
Exposes = Exposes + href_extr(hrefs)
Projekte = Projekte + projekt_finder(hrefs)

In [8]:
for a in range(2,num_pages+1):
    new_url = url+'&cp='+str(a)
    soup_new = soup_get(new_url)
    href_new = href_finder(soup_new)
    Exposes = Exposes+href_extr(href_new)
    Projekte = Projekte + projekt_finder(href_new)


In [11]:
a +=1
new_url = url+'&cp='+str(a)
soup_new = soup_get(new_url)
href_new = href_finder(soup_new)
Exposes = Exposes+href_extr(href_new)
Projekte = Projekte + projekt_finder(href_new)

In [20]:
with open('Expose.txt','w') as f:
    f.write(str(Exposes))

An alternative way of looping through pages:
get element on which to click

```python
text = driver.find_element_by_id('nlbPlus')
```


click element 
```
text.click()
```

switch selenium driver to read new page
```
driver.switch_to_window(driver.window_handles[0]) 
```